In [23]:
from env import config, BASE_DIR
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker


In [2]:
DATABASE_URL = config("FLIGHT_PRICES_DATABASE_URL", cast=str, default=None)
DATABASE_URL is not None

True

In [3]:
engine = create_engine(str(DATABASE_URL))

In [4]:
DATASET_DIR = BASE_DIR / "datasets"
DATASET_DIR.exists()

True

In [5]:
FLIGHT_PRICES_CSV = DATASET_DIR / "itineraries-min-50k.csv"
FLIGHT_PRICES_CSV.exists()

True

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv(str(FLIGHT_PRICES_CSV))
df.head(10)

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H29M,0.0,False,False,...,1650223560,2022-04-17T15:26:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,8940,947,coach
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0.0,False,False,...,1650200400,2022-04-17T09:00:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9000,947,coach
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0.0,False,False,...,1650218700,2022-04-17T14:05:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757-200,9000,947,coach
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H32M,0.0,False,False,...,1650227460,2022-04-17T16:31:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9120,947,coach
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H34M,0.0,False,False,...,1650213180,2022-04-17T12:33:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9240,947,coach
5,79eda9f841e226a1e2121d74211e595c,2022-04-16,2022-04-17,ATL,BOS,VH0AUEL1,PT2H38M,0.0,False,False,...,1650216180,2022-04-17T13:23:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9480,947,coach
6,9335fae376c38bb61263281779f469ec,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT4H12M,0.0,False,False,...,1650203400||1650213120,2022-04-17T09:50:00.000-04:00||2022-04-17T12:3...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A320||Airbus A320,5400||7500,228||728,coach||coach
7,3904bf87f2d1daf334f1ae7e3b876028,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT5H18M,0.0,False,False,...,1650203400||1650217080,2022-04-17T09:50:00.000-04:00||2022-04-17T13:3...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A320||Boeing 737-800,5400||8280,228||728,coach||coach
8,d93988734c44a3c075d9efe373352507,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT5H32M,0.0,False,False,...,1650198060||1650213120,2022-04-17T08:21:00.000-04:00||2022-04-17T12:3...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A319||Airbus A320,4860||7500,228||728,coach||coach
9,562e7d5dd6ecbf1509c0c19711dbdca9,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT6H38M,0.0,False,False,...,1650198060||1650217080,2022-04-17T08:21:00.000-04:00||2022-04-17T13:3...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A319||Boeing 737-800,4860||8280,228||728,coach||coach


In [8]:
startingAirports = set(df.startingAirport.unique().tolist())

In [9]:
destinationAirports = set(df.destinationAirport.unique().tolist())

In [10]:
missing = startingAirports - destinationAirports
missing

set()

In [11]:
import airportsdata
airports = airportsdata.load('IATA')

In [12]:
avalible_airports = [airports[x] for x in startingAirports]
avalible_airports

[{'icao': 'KMIA',
  'iata': 'MIA',
  'name': 'Miami International Airport',
  'city': 'Miami',
  'subd': 'Florida',
  'country': 'US',
  'elevation': 9.3,
  'lat': 25.7953611111111,
  'lon': -80.2901158333333,
  'tz': 'America/New_York',
  'lid': 'MIA'},
 {'icao': 'KDTW',
  'iata': 'DTW',
  'name': 'Detroit Metro Wayne County Airport',
  'city': 'Detroit',
  'subd': 'Michigan',
  'country': 'US',
  'elevation': 645.2,
  'lat': 42.2124444444445,
  'lon': -83.3533888888889,
  'tz': 'America/New_York',
  'lid': 'DTW'},
 {'icao': 'KPHL',
  'iata': 'PHL',
  'name': 'Philadelphia International Airport',
  'city': 'Philadelphia',
  'subd': 'Pennsylvania',
  'country': 'US',
  'elevation': 35.9,
  'lat': 39.8720838888889,
  'lon': -75.2406630555556,
  'tz': 'America/New_York',
  'lid': 'PHL'},
 {'icao': 'KDFW',
  'iata': 'DFW',
  'name': 'Dallas-Fort Worth International Airport',
  'city': 'Dallas-Fort Worth',
  'subd': 'Texas',
  'country': 'US',
  'elevation': 606.4,
  'lat': 32.897233055555

In [13]:
airports_df = pd.DataFrame(avalible_airports)

In [14]:
airports_df.dtypes

icao          object
iata          object
name          object
city          object
subd          object
country       object
elevation    float64
lat          float64
lon          float64
tz            object
lid           object
dtype: object

In [16]:
table_name = "airports"
airports_df.to_sql(table_name, engine, if_exists='replace', index=True)

16

In [27]:
from sqlalchemy import Column, BigInteger, Date, Integer, String, Boolean, Float
from sqlalchemy.orm import DeclarativeBase


class Base(DeclarativeBase):
    pass


class Airport(Base):
    __tablename__ = "airports"
    __table_args__ = {"extend_existing": True}
    
    id = Column("index", BigInteger, primary_key=True)
    name = Column(String)
    icao = Column(String)
    iata = Column(String)
    city = Column(String)
    subd = Column(String)
    country = Column(String)
    elevation = Column(Float)
    lat = Column(Float)
    lon = Column(Float)
    tz = Column(String)
    lid = Column(String)
    

In [25]:
from pydantic import BaseModel, Field


class AirportSchema(BaseModel):
    id: int
    label: str = Field(alias="name")
    value: str = Field(alias="iata")

    class Config:
        from_attributes = True

In [28]:
SessionLocal = sessionmaker(bind=engine)
session = SessionLocal()

queryset = session.query(Airport).limit(10).all()
for row_value in queryset:
    data = AirportSchema.model_validate(row_value)
    print(data.model_dump_json())

session.close()

{"id":0,"label":"Miami International Airport","value":"MIA"}
{"id":1,"label":"Detroit Metro Wayne County Airport","value":"DTW"}
{"id":2,"label":"Philadelphia International Airport","value":"PHL"}
{"id":3,"label":"Dallas-Fort Worth International Airport","value":"DFW"}
{"id":4,"label":"Newark Liberty International Airport","value":"EWR"}
{"id":5,"label":"Chicago O'Hare International Airport","value":"ORD"}
{"id":6,"label":"Hartsfield - Jackson Atlanta International Airport","value":"ATL"}
{"id":7,"label":"John F Kennedy International Airport","value":"JFK"}
{"id":8,"label":"Charlotte/Douglas International Airport","value":"CLT"}
{"id":9,"label":"Laguardia Airport","value":"LGA"}


In [8]:
df['flightDate'] = pd.to_datetime(df['flightDate'])

In [9]:
group_by_columns = ['flightDate', 'startingAirport', 'destinationAirport', 'isBasicEconomy',  'isRefundable', 'isNonStop', 'segmentsAirlineName']
target_column = 'totalFare'
desired_columns = group_by_columns + [target_column]
sub_df = df.copy()[desired_columns]
sub_df = sub_df.groupby(group_by_columns)[target_column].mean().reset_index()
sub_df.head()

,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,segmentsAirlineName,totalFare
0,2022-04-17,ATL,BOS,False,False,False,American Airlines||American Airlines,272.729167
1,2022-04-17,ATL,BOS,False,False,False,Spirit Airlines||Spirit Airlines,301.080000
2,2022-04-17,ATL,BOS,False,False,False,United||United,352.457143
3,2022-04-17,ATL,BOS,False,False,True,American Airlines,355.140000
4,2022-04-17,ATL,BOS,False,False,True,Delta,307.488889


In [10]:
sub_df[target_column] = sub_df[target_column].round(0).apply(lambda x: int(x))
sub_df.dropna(inplace=True)
sub_df.head()

,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,segmentsAirlineName,totalFare
0,2022-04-17,ATL,BOS,False,False,False,American Airlines||American Airlines,273
1,2022-04-17,ATL,BOS,False,False,False,Spirit Airlines||Spirit Airlines,301
2,2022-04-17,ATL,BOS,False,False,False,United||United,352
3,2022-04-17,ATL,BOS,False,False,True,American Airlines,355
4,2022-04-17,ATL,BOS,False,False,True,Delta,307


In [11]:
sub_df['startingAirport'] = sub_df['startingAirport'].apply(lambda x: f"{x}".strip())
sub_df['destinationAirport'] = sub_df['destinationAirport'].apply(lambda x: f"{x}".strip())

In [12]:
out_df = df.copy()#[['date', 'price', 'type', 'bedrooms']]

In [13]:
out_df = sub_df.copy()

In [14]:
table_name = "flight_prices"
out_df.to_sql(table_name, engine, if_exists='replace', index=True)

9592

In [16]:
with engine.connect() as conn:
    results = conn.execute(text(f"DESCRIBE {table_name}"))
    print(results.all())

[('index', 'bigint(20)', 'YES', 'MUL', None, ''), ('flightDate', 'datetime', 'YES', '', None, ''), ('startingAirport', 'text', 'YES', '', None, ''), ('destinationAirport', 'text', 'YES', '', None, ''), ('isBasicEconomy', 'tinyint(1)', 'YES', '', None, ''), ('isRefundable', 'tinyint(1)', 'YES', '', None, ''), ('isNonStop', 'tinyint(1)', 'YES', '', None, ''), ('segmentsAirlineName', 'text', 'YES', '', None, ''), ('totalFare', 'bigint(20)', 'YES', '', None, '')]


In [20]:
out_df.dtypes

flightDate             datetime64[ns]
startingAirport                object
destinationAirport             object
isBasicEconomy                   bool
isRefundable                     bool
isNonStop                        bool
segmentsAirlineName            object
totalFare                       int64
dtype: object